# Chapter 1

In [1]:
import pandas as pd
import numpy as np

url = 'https://assets.datacamp.com/production/repositories/5286/datasets/45e193467da41ae7631b0d4d626c63d832a34cab/names.txt'
names_df = pd.read_csv(url, header=None, names=['input'])
names_df

,input
0,John
1,William
2,James
3,Charles
4,George
...,...
257995,Carleigh
257996,Iyana
257997,Kenley
257998,Sloane


In [2]:
# Insert a tab in front of all the names
names_df['input'] = names_df['input'].apply(lambda x : '\t' + x)

# Append a newline at the end of every name
# We already appended a tab in front, so the target word should start at index 1
names_df['target'] = names_df['input'].apply(lambda x : x[1:len(x)] + '\n')

In [3]:
def get_vocabulary(word_list):
    char_set = set('\n') # loop removes newline, add to set
    for word in word_list:
        for char in word:
            char_set.add(char)
    return char_set

In [4]:
# Get the vocabulary
vocabulary = get_vocabulary(names_df['input'])

# Sort the vocabulary
vocabulary_sorted = sorted(vocabulary)

# Create the mapping of the vocabulary chars to integers
char_to_idx = { char : idx for idx, char in enumerate(vocabulary_sorted) }

# Create the mapping of the integers to vocabulary chars
idx_to_char = { idx : char for idx, char in enumerate(vocabulary_sorted) }

# Print the dictionaries
print(char_to_idx)
print(idx_to_char)

{'\t': 0, '\n': 1, 'A': 2, 'B': 3, 'C': 4, 'D': 5, 'E': 6, 'F': 7, 'G': 8, 'H': 9, 'I': 10, 'J': 11, 'K': 12, 'L': 13, 'M': 14, 'N': 15, 'O': 16, 'P': 17, 'Q': 18, 'R': 19, 'S': 20, 'T': 21, 'U': 22, 'V': 23, 'W': 24, 'X': 25, 'Y': 26, 'Z': 27, 'a': 28, 'b': 29, 'c': 30, 'd': 31, 'e': 32, 'f': 33, 'g': 34, 'h': 35, 'i': 36, 'j': 37, 'k': 38, 'l': 39, 'm': 40, 'n': 41, 'o': 42, 'p': 43, 'q': 44, 'r': 45, 's': 46, 't': 47, 'u': 48, 'v': 49, 'w': 50, 'x': 51, 'y': 52, 'z': 53}
{0: '\t', 1: '\n', 2: 'A', 3: 'B', 4: 'C', 5: 'D', 6: 'E', 7: 'F', 8: 'G', 9: 'H', 10: 'I', 11: 'J', 12: 'K', 13: 'L', 14: 'M', 15: 'N', 16: 'O', 17: 'P', 18: 'Q', 19: 'R', 20: 'S', 21: 'T', 22: 'U', 23: 'V', 24: 'W', 25: 'X', 26: 'Y', 27: 'Z', 28: 'a', 29: 'b', 30: 'c', 31: 'd', 32: 'e', 33: 'f', 34: 'g', 35: 'h', 36: 'i', 37: 'j', 38: 'k', 39: 'l', 40: 'm', 41: 'n', 42: 'o', 43: 'p', 44: 'q', 45: 'r', 46: 's', 47: 't', 48: 'u', 49: 'v', 50: 'w', 51: 'x', 52: 'y', 53: 'z'}


In [5]:
def get_max_len(words):
    return max([len(word) for word in words])
    
get_max_len(['1', '12', '\t234\n'])

5

In [6]:
vocabulary = char_to_idx

In [7]:
# Find the length of longest name
max_len = get_max_len(names_df['input'])

# Initialize the input vector
input_data = np.zeros((len(names_df['input']), max_len+1, len(vocabulary)), dtype='float32')

# Initialize the target vector
target_data = np.zeros((len(names_df['input']), max_len+1, len(vocabulary)), dtype='float32')

In [8]:
# I thought target would be a single character
input_data.shape, target_data.shape

((258000, 13, 54), (258000, 13, 54))

In [9]:
# Iterate for each name in the dataset
for n_idx, name in enumerate(names_df['input']):
  # Iterate over each character and convert it to a one-hot encoded vector
  for c_idx, char in enumerate(name):
    input_data[n_idx, c_idx, char_to_idx[char]] = 1

# Iterate for each name in the dataset
for n_idx, name in enumerate(names_df['target']):
  # Iterate over each character and convert it to a one-hot encoded vector
  for c_idx, char in enumerate(name):
    target_data[n_idx, c_idx, char_to_idx[char]] = 1

In [10]:
from keras.models import Sequential
from keras.layers import SimpleRNN, TimeDistributed, Dense

# The model build would not work until I downgraded to numpy
# cond install numpy=1.19

In [11]:
# Create a Sequential model
model = Sequential()

# Add SimpleRNN layer of 50 units
model.add(SimpleRNN(50, input_shape=(max_len+1, len(vocabulary)), return_sequences=True))

# Add a TimeDistributed Dense layer of size same as the vocabulary
model.add(TimeDistributed(Dense(len(vocabulary), activation='softmax')))

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam")

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 13, 50)            5250      
_________________________________________________________________
time_distributed (TimeDistri (None, 13, 54)            2754      
Total params: 8,004
Trainable params: 8,004
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Fit the model for 5 epochs using a batch size of 128 
model.fit(input_data, target_data, batch_size=128, epochs=5)

# Create a 3-D zero vector and initialize it with the start token
output_seq = np.zeros((1, max_len+1, len(vocabulary)))
output_seq[0, 0, char_to_idx['\t']] = 1

Epoch 1/5
2016/2016 [==============================] - 56s 27ms/step - loss: 1.2902
Epoch 2/5
2016/2016 [==============================] - 52s 26ms/step - loss: 1.0155
Epoch 3/5
2016/2016 [==============================] - 50s 25ms/step - loss: 0.9710
Epoch 4/5
2016/2016 [==============================] - 61s 30ms/step - loss: 0.9448
Epoch 5/5
2016/2016 [==============================] - 70s 35ms/step - loss: 0.9257


In [16]:
# Create a 3-D zero vector and initialize it with the start token
output_seq = np.zeros((1, max_len+1, len(vocabulary)))
output_seq[0, 0, char_to_idx['\t']] = 1

In [17]:
# Get the probabilities for the first character
probs = model.predict_proba(output_seq, verbose=0)[:,1,:]

# Sample vocabulary to get first character
first_char = np.random.choice(sorted(vocabulary), replace=False, p=probs.reshape(len(vocabulary)))

# Print the character generated
print(first_char)

/home/bosullivan/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


a


In [18]:
# Print the first character which we got last time
print(first_char)

# Update the vector to contain first the character
output_seq[0, 1, char_to_idx[first_char]] = 1

# Get the probabilities for the second character
probs = model.predict_proba(output_seq, verbose=0)[:,2,:]

# Sample vocabulary to get second character
second_char = np.random.choice(sorted(vocabulary), replace=False, p=probs.reshape(len(vocabulary)))

# Print the second character
print(second_char)

a
n


In [ ]:
# Get the probabilities for the second character
probs = model.predict_proba(output_seq, verbose=0)[:,2,:]

# Sample vocabulary to get second character
second_char = np.random.choice(sorted(vocabulary), replace=False, p=probs.reshape(len(vocabulary)))

In [20]:
def generate_baby_names(n):
    # return n baby names
    for i in range(n):
        stop = False
        counter = 1
        name = ''
        # Create a 3-D zero vector and initialize it with the start token
        output_seq = np.zeros((1, max_len+1, len(vocabulary)))
        output_seq[0, 0, char_to_idx['\t']] = 1
        while not stop and counter < 10:
            # Get the probabilities for the next character
            probs = model.predict_proba(output_seq, verbose=0)[:,counter-1,:]
            c = np.random.choice(sorted(vocabulary), replace=False, p=probs.reshape(len(vocabulary)))
            if c == '\n':
                stop = True
            else:
                name = name + c
                output_seq[0, counter, char_to_idx[c]] = 1
                counter = counter + 1
        print(name)

In [ ]:
generate_baby_names(10)
